In [8]:
library(Seurat)
library(ggplot2)
library(mltools)
library(data.table)
library(lsa)
library(aricode)
library(kBET)
library(cowplot)
options(repr.plot.width=14, repr.plot.height=10)
library(viridis)
library(mclust)

Loading required package: SnowballC


Attaching package: ‘aricode’


The following object is masked from ‘package:lsa’:

    entropy


Loading required package: viridisLite

Package 'mclust' version 5.4.7
Type 'citation("mclust")' for citing this R package in publications.



In [2]:
seur <- readRDS('/gpfs/ysm/pi/zhao-data/wd262/new_cytof/write/BCR/5celltype/seur_patient1_BCR-XL.rds')
rst <- readRDS('/gpfs/ysm/pi/zhao-data/wd262/new_cytof/write/BCR/5celltype/rst_patient1_BCR-XL.rds')
truth <- seur$label

In [3]:
celltype_pred <- seur$pred
H_est_subsetting <- data.frame(rst$H[,-1])
H_est_subsetting <- apply(H_est_subsetting,2,function(x) x/sum(x))

In [4]:
H_est_subsetting

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X2742,X2743,X2744,X2745,X2746,X2747,X2748,X2749,X2750,X2751
1.369773e-20,8.838661e-44,1.509824e-01,7.577876e-02,4.448538e-02,3.776206e-01,1.453928e-20,2.025539e-20,5.657829e-01,1.731321e-01,⋯,2.086555e-10,0.1355811183,1.660717e-08,0.0003816343,2.527837e-14,1.618384e-13,7.845042e-10,1.735769e-21,4.499247e-23,1.083760e-01
9.996456e-01,9.999999e-01,6.570936e-06,6.245497e-01,1.368873e-07,3.663868e-07,1.338265e-01,8.364615e-07,3.701299e-06,9.982636e-07,⋯,3.855504e-03,0.0006525962,3.873510e-04,0.0005537549,7.204084e-03,1.564082e-01,5.070751e-01,9.379093e-01,8.626875e-01,7.039129e-05
3.544442e-04,5.341121e-08,4.789615e-02,2.614589e-01,4.468719e-01,6.223790e-01,7.946710e-01,1.848684e-01,3.510674e-01,4.939104e-01,⋯,7.533872e-01,0.3772920819,6.136970e-01,0.9155889000,4.401680e-01,3.270337e-01,2.380659e-24,1.747884e-05,1.021040e-01,1.182769e-01
2.044086e-20,9.496626e-42,3.698003e-02,3.821259e-02,7.147210e-04,4.486101e-10,3.414172e-21,2.475241e-30,7.847832e-19,3.329565e-01,⋯,1.694837e-08,0.1633922404,7.909351e-09,0.0020237713,2.573060e-08,9.526172e-02,3.828379e-01,5.890065e-18,2.218875e-28,5.889483e-02
2.250318e-15,4.646858e-09,7.641348e-01,6.461229e-27,5.079279e-01,1.560718e-09,7.150256e-02,8.151307e-01,8.314598e-02,1.103057e-19,⋯,2.427573e-01,0.3230819632,3.859156e-01,0.0814519395,5.526279e-01,4.212963e-01,1.100870e-01,6.207319e-02,3.520853e-02,7.143819e-01


In [5]:
rst$H

"matrix(colnames(A0), ncol = 1)",X1,X2,X3,X4,X5,X6,X7,X8,X9,⋯,X2742,X2743,X2744,X2745,X2746,X2747,X2748,X2749,X2750,X2751
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
naïve B cells,1.401965e-20,9.394162e-44,1.546003e-01,7.878238e-02,4.492856e-02,3.845289e-01,1.481032e-20,2.130327e-20,5.887215e-01,⋯,2.107812e-10,0.1351595318,1.678019e-08,0.0003822888,2.529056e-14,1.645890e-13,7.780065e-10,1.795140e-21,4.690928e-23,1.098089e-01
CD4 T-cells,1.023139e+00,1.062849e+00,6.728391e-06,6.493048e-01,1.382510e-07,3.730896e-07,1.363212e-01,8.797347e-07,3.851361e-06,⋯,3.894783e-03,0.0006505669,3.913865e-04,0.0005547047,7.207559e-03,1.590665e-01,5.028752e-01,9.699897e-01,8.994405e-01,7.132196e-05
CD8 T-cells,3.627743e-04,5.676805e-08,4.904385e-02,2.718223e-01,4.513237e-01,6.337650e-01,8.094852e-01,1.944323e-01,3.653007e-01,⋯,7.610626e-01,0.3761189004,6.200906e-01,0.9171592743,4.403804e-01,3.325920e-01,2.360941e-24,1.807669e-05,1.064539e-01,1.198407e-01
monocytes,2.092126e-20,1.009348e-41,3.786616e-02,3.972721e-02,7.218413e-04,4.568171e-10,3.477819e-21,2.603294e-30,8.166007e-19,⋯,1.712103e-08,0.1628841758,7.991752e-09,0.0020272424,2.574302e-08,9.688078e-02,3.796670e-01,6.091530e-18,2.313405e-28,5.967350e-02
NK cells,2.303204e-15,4.938908e-09,7.824453e-01,6.717331e-27,5.129880e-01,1.589270e-09,7.283551e-02,8.573003e-01,8.651698e-02,⋯,2.452305e-01,0.3220773469,3.899361e-01,0.0815916420,5.528945e-01,4.284566e-01,1.091752e-01,6.419635e-02,3.670852e-02,7.238270e-01


In [9]:
truth_fact_sub <- as.factor(truth)
truth_onehot_sub <- as.data.frame(t(one_hot(as.data.table(truth_fact_sub))))
truth_onehot_sub

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V2742,V2743,V2744,V2745,V2746,V2747,V2748,V2749,V2750,V2751
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
truth_fact_sub_CD4 T-cells,1,1,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,1,1,1,0
truth_fact_sub_CD8 T-cells,0,0,0,0,1,1,1,0,1,1,⋯,1,1,1,1,0,0,0,0,0,0
truth_fact_sub_monocytes,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
truth_fact_sub_naïve B cells,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
truth_fact_sub_NK cells,0,0,1,0,0,0,0,1,0,0,⋯,0,0,0,0,1,1,0,0,0,1
